In [1]:
from scipy.stats import pearsonr, spearmanr
# import tensorflow as tf
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import itertools

In [2]:
def trim_and_calculate_mean(array_list):
    # Find the minimum length among all arrays
    min_length = min(len(arr) for arr in array_list)
    
    # Trim each array to the minimum length
    trimmed_arrays = [arr[:min_length] for arr in array_list]
    
    # Convert to a NumPy array for vectorized mean calculation
    trimmed_arrays = np.array(trimmed_arrays)
    
    return trimmed_arrays

In [39]:
folders = [
    # 'test_study1__dynamic_stacked_view_simple_actions_3agents_respawn_0.04_v0_s5_seed2_20250312-14031741802613',
    'test_study1__dynamic_with_cost0.2_stacked_view_simple_actions_3agents_respawn_0.04_v0_s1_seed2_20250312-00031741752223',
    # 'test_study1__nondynamic_cost0.2_stacked_view_simple_actions_3agents_respawn_0.04_v0_s1_seed1_20250311-23031741751523',
    # 'test_study1__nondynamic_stacked_view_simple_actions_3agents_respawn_0.04_v0_s1_seed1_20250307-11031741366516',
    # 'test_study1__fixed_only_taboo_stacked_view_simple_actions_3agents_respawn_0.04_v0_s4_seed1_20250314-13031741973313'
]

collective_authoritarianism = [[] for _ in range(len(folders))]
collective_transgressions = [[] for _ in range(len(folders))]
collective_d_prime = [[] for _ in range(len(folders))]
collective_coin = [[] for _ in range(len(folders))]
collective_gem= [[] for _ in range(len(folders))]

for ixs, folder in enumerate(folders):
    parent_dir = os.path.join('res', folder)
    files = os.listdir(parent_dir)

    for f in files:
        if 'state_punishment_level_avg' in f:
        # if 'd_prime' in f:
            data = pd.read_csv(os.path.join(parent_dir, f))
            collective_authoritarianism[ixs].append(data['Value'].to_numpy()[:])
        if 'Bone' in f:
            sub_dir = os.path.join(parent_dir, f)
            file = os.listdir(sub_dir)[0]
            data = pd.read_csv(os.path.join(sub_dir, file))
            collective_transgressions[ixs].append(data['Value'].to_numpy()[:])
        if 'Gem' in f:
            sub_dir = os.path.join(parent_dir, f)
            file = os.listdir(sub_dir)[0]
            data = pd.read_csv(os.path.join(sub_dir, file))
            collective_gem[ixs].append(data['Value'].to_numpy()[:])
        if 'Coin' in f:
            sub_dir = os.path.join(parent_dir, f)
            file = os.listdir(sub_dir)[0]
            data = pd.read_csv(os.path.join(sub_dir, file))
            collective_coin[ixs].append(data['Value'].to_numpy()[:])
        if 'd_prime' in f:
            data = pd.read_csv(os.path.join(parent_dir, f))
            collective_d_prime[ixs].append(data['Value'].to_numpy()[:])
collective_authoritarianism_ = [np.mean(trim_and_calculate_mean(collective_authoritarianism[i]), axis=0) 
                  for i in range(len(collective_authoritarianism))]
collective_transgressions_ = [np.mean(trim_and_calculate_mean(collective_transgressions[i]), axis=0) 
                  for i in range(len(collective_transgressions))]
collective_d_prime_ = [np.mean(trim_and_calculate_mean(collective_d_prime[i]), axis=0) 
                  for i in range(len(collective_d_prime))]
collective_gem_ = [np.mean(trim_and_calculate_mean(collective_gem[i]), axis=0) 
                  for i in range(len(collective_gem))]
collective_coin_ = [np.mean(trim_and_calculate_mean(collective_coin[i]), axis=0) 
                  for i in range(len(collective_coin))]
print(collective_authoritarianism_, len(collective_authoritarianism_[0]))
print(collective_transgressions_, len(collective_transgressions_[0]))

[array([0.74400002, 0.018     , 0.        , ..., 0.90200001, 0.94999999,
       0.93000001])] 144002
[array([1.66666667, 0.66666667, 0.66666667, ..., 0.        , 0.33333333,
       0.        ])] 144002


In [45]:
endpoint = 98000
folder_ixs = 0
diff_authoritarianism = np.diff(collective_authoritarianism_[folder_ixs][80000:endpoint])
diff_d_prime = np.diff(collective_d_prime_[folder_ixs][80000:endpoint])
# print(len(diff_authoritarianism), collective_transgressions_[folder_ixs][:endpoint].shape)

print('transgression ~ diff authoritarianism')
print(spearmanr(collective_transgressions_[folder_ixs][80000:endpoint-1], diff_authoritarianism),'\n');

print('transgression ~ d-prime diff')
print(spearmanr(collective_transgressions_[folder_ixs][80000:endpoint-1], diff_d_prime), '\n');

print('coin ~ d-prime diff')
print(spearmanr(collective_coin_[folder_ixs][80000:endpoint-1], diff_d_prime), '\n');

print('authoritarianism ~ diff of authoritarianism')
print(spearmanr(collective_authoritarianism_[folder_ixs][80000:endpoint-1], diff_authoritarianism))


transgression ~ diff authoritarianism
SignificanceResult(statistic=np.float64(0.13764113609222062), pvalue=np.float64(7.703298662298796e-77)) 

transgression ~ d-prime diff
SignificanceResult(statistic=np.float64(-0.31403109751666214), pvalue=np.float64(0.0)) 

coin ~ d-prime diff
SignificanceResult(statistic=np.float64(0.020205122905097423), pvalue=np.float64(0.006711809472815495)) 

authoritarianism ~ diff of authoritarianism
SignificanceResult(statistic=np.float64(-0.4331700607752067), pvalue=np.float64(0.0))


In [35]:
collective_authoritarianism_[folder_ixs][80000:endpoint-1]

array([0.95599997, 0.69199997, 0.722     , ..., 0.86799997, 0.76800001,
       0.86400002])

In [37]:
0.69199997-0.95599997

-0.264

In [36]:
diff_authoritarianism

array([-0.264     ,  0.03000003,  0.13      , ..., -0.09999996,
        0.09600002,  0.02199996])

In [23]:
print('transgression+good acts ~ d-prime diff')
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Sample data
data = {
    'X1': collective_transgressions_[folder_ixs][80000:endpoint-1],
    'X2': collective_coin_[folder_ixs][80000:endpoint-1],
    'X3': collective_gem_[folder_ixs][80000:endpoint-1],
    'Y': diff_d_prime
}

df = pd.DataFrame(data)

# Define independent variables (X) and dependent variable (Y)
X = df[['X1', 'X2', 'X3']]  # Independent variables
Y = df['Y']           # Dependent variable

# Create the regression model
model = LinearRegression()

# Fit the model on the entire dataset
model.fit(X, Y)

# Make predictions on the same data
Y_pred = model.predict(X)

# Evaluate the model
mse = mean_squared_error(Y, Y_pred)
r2 = r2_score(Y, Y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

# Print coefficients and intercept
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)

transgression+good acts ~ d-prime diff
Mean Squared Error: 15.018904275187259
R^2 Score: 0.0010418055795377024
Intercept: 0.7432842541369222
Coefficients: [ 0.07053319 -0.03531614 -0.00272125]
